In [1]:
# Create custom dataset. Idea supplied by Mat Cook

from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [2]:
import collections
from skimage.color import rgb2gray
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [3]:
numpy_data = np.load('train_data.npy')
numpy_target = np.load('train_labels.npy')
numpy_data = numpy_data
numpy_data = numpy_data.transpose((0, 3, 2, 1))
le = LabelEncoder()
le.fit(numpy_target)
target = le.transform(numpy_target)
target = target+1

X_train, X_test, y_train, y_test = train_test_split(numpy_data, target, test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1475, 3, 100, 100), (369, 3, 100, 100), (1475,), (369,))

In [ ]:
# Tried converting to tensor here but it doesn't seem to be the best approach
Xtrain = torch.from_numpy(X_train).float()
Xtest = torch.from_numpy(X_test).float()
Xtrain.shape, Xtest.shape

In [4]:
# Creat Custom dataset. This is supposed to transfom the data into pythorch friendly data
class MyDataset(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).float()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
        
        return x, y
    
    def __len__(self):
        return len(self.data)
    

In [ ]:
# class ToTensor(object):
#     """Convert ndarrays in sample to Tensors."""

#     def __call__(self, sample):
#         # swap color axis because
#         # numpy image: H x W x C
#         # torch image: C X H X W
#         sample = torch.from_numpy(sample)

In [5]:
# Initialize parameters. These will likely need to be tuend
n_epochs = 3
batch_size_train = 4
batch_size_test = 175
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [6]:
# Create data using Dataloader. I think I'm supposed to use dataset here as well,
# little unsure how to do that tho. 
dataset_train = MyDataset(X_train, y_train)
loader = torch.utils.data.DataLoader(
    dataset_train, 
    batch_size=batch_size_train,
    shuffle=True,
    )
    
dataset_test = MyDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=batch_size_test,
    shuffle=True,
)

In [ ]:
# dataset returnds dictionary {}

In [ ]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape, example_targets.shape

In [ ]:
examples = enumerate(loader)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape, example_targets.shape

In [ ]:
# import matplotlib.pyplot as plt

# fig = plt.figure()
# for i in range(6):
#   plt.subplot(2,3,i+1)
#   plt.tight_layout()
#   plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
#   plt.title("Ground Truth: {}".format(example_targets[i]))
#   plt.xticks([])
#   plt.yticks([])
# fig

In [ ]:
data, label = dataset_test

In [34]:
# Create model, pretty sure the dimension are wrong in all fo these
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=0)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3)
        self.fc1 = nn.Linear(30000,  5000)
        self.fc2 = nn.Linear(5000, 2)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16928)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [35]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [36]:
 # run through model
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

RuntimeError: size mismatch, m1: [1 x 16928], m2: [30000 x 5000] at C:\w\1\s\tmp_conda_3.7_075911\conda\conda-bld\pytorch_1579075223148\work\aten\src\TH/generic/THTensorMath.cpp:136

In [ ]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(loader.dataset) for i in range(n_epochs + 1)]

In [ ]:
# can alsow run through model this way
def train(epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(loader):
        optimizer.zero_grad()
        output = network(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(loader.dataset),
                100. * batch_idx / len(loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append(
                (batch_idx*64) + ((epoch-1)*len(loader.dataset)))
#             torch.save(network.state_dict(),'\results\model.pth')
#             torch.save(optimizer.state_dict(),'\results\optimizer.pth')

In [ ]:
def test():
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [ ]:
test()
for epoch in range(1, n_epochs + 1):
    train(epoch)
    test()

In [ ]:
fig = plt.figure()
plt.plot(train_counter, train_losses, color='blue')
plt.scatter(test_counter, test_losses, color='red')
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss')
fig

In [ ]:
np.shape(test_counter), np.shape(test_losses)

In [ ]:
fig = plt.figure(figsize=(5,4))
for i in range(10):
    ax = fig.add_subplot(4,3,i+1)
    ax.imshow(X_train[np.where(y_train==i)[0][1],:].reshape((28,28)), cmap='binary')

In [ ]:
data, target = test_loader

In [ ]:
img = torch.tensor(X_test[200,:,:,:])
img.shape

In [ ]:
# can also run through model this way
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(3125000, 100)
        self.fc2 = nn.Linear(100, 2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1).float()
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [ ]:
model = Net()
model(img.unsqueeze(0))
# everything past this point is just a confused mess. 

In [ ]:
import datetime 

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):  
        loss_train = 0.0
        for imgs, labels in train_loader: 
            outputs = model(imgs)  
            loss = loss_fn(outputs, labels) 

            optimizer.zero_grad() 
            loss.backward() 
            optimizer.step()

            loss_train += loss.item() 

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch, loss_train / len(train_loader)))

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=625, shuffle=True) 

model = Net()
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

In [18]:
connected_model = nn.Sequential(
    nn.Linear(30000,10000), nn.Tanh(),
    nn.Linear(10000, 5000), nn.Tanh(),
    nn.Linear(5000, 2500), nn.Tanh(),
    nn.Linear(2500, 1250))

In [19]:
numel_list = [p.numel() for p in connected_model.parameters() if p.requires_grad==True]
sum(numel_list), numel_list

(365643750, [300000000, 10000, 50000000, 5000, 12500000, 2500, 3125000, 1250])

In [20]:
connected_model = nn.Sequential(
    nn.Linear(30000,5000), nn.Tanh(),
    nn.Linear(5000, 2), nn.LogSoftmax(dim=1))

In [21]:
numel_list = [p.numel() for p in connected_model.parameters() if p.requires_grad==True]
sum(numel_list), numel_list

(150015002, [150000000, 5000, 10000, 2])

In [23]:
conv = nn.Conv2d(3, 16, kernel_size = 3, padding= 0)
conv

Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))

In [24]:
conv.weight.shape,  conv.bias.shape


(torch.Size([16, 3, 3, 3]), torch.Size([16]))

In [25]:
img, _ = loader

ValueError: too many values to unpack (expected 2)